In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 

In [ ]:

loan = pd.read_excel("../input/loan-approval-analysis/Loan Data/loan.xlsx")
loan.info()

In [ ]:
# let's look at the first few rows of the df
loan.head()

In [ ]:

loan.columns

In [ ]:
#missing values in each column
loan.isnull().sum()

In [ ]:
# percentage of missing values in each column
round(loan.isnull().sum()/len(loan.index), 2)*100

In [ ]:
# removing the columns having more than 90% missing values
missing_columns = loan.columns[100*(loan.isnull().sum()/len(loan.index)) > 90]
print(missing_columns)

In [ ]:
loan = loan.drop(missing_columns, axis=1)
print(loan.shape)


In [ ]:
# summarise number of missing values again
100*(loan.isnull().sum()/len(loan.index))

In [ ]:
# tow columns have 32 and 64% of mising values 
loan.loc[:, ['desc', 'mths_since_last_delinq']].head()

In [ ]:
# this 2 columns are added after created after the loan approved
loan = loan.drop(['desc', 'mths_since_last_delinq'], axis=1)

In [ ]:
# summarise number of missing values again
100*(loan.isnull().sum()/len(loan.index))

In [ ]:
# missing values in rows
loan.isnull().sum(axis=1)

In [ ]:
# checking whether some rows have more than 5 missing values
len(loan[loan.isnull().sum(axis=1) > 5].index)

In [ ]:
loan.info()

In [ ]:
# The column int_rate is character type, let's convert it to float
loan['int_rate'] = loan['int_rate'].astype(str).str.split()


In [ ]:
# checking the data types
loan.info()

In [ ]:
# also, lets extract the numeric part from the variable employment length

# first, let's drop the missing values from the column (otherwise the regex code below throws error)
loan = loan[~loan['emp_length'].isnull()]

# using regular expression to extract numeric values from the string
import re
loan['emp_length'] = loan['emp_length'].apply(lambda x: re.findall('\d+', str(x))[0])

# convert to numeric
loan['emp_length'] = loan['emp_length'].apply(lambda x: pd.to_numeric(x))

In [ ]:
# looking at type of the columns again
loan.info()

The objective is to identify predictors of default so that at the time of loan application, we can use those variables for approval/rejection of the loan

In [ ]:
behaviour_var =  [
  "delinq_2yrs",
  "earliest_cr_line",
  "inq_last_6mths",
  "open_acc",
  "pub_rec",
  "revol_bal",
  "revol_util",
  "total_acc",
  "out_prncp",
  "out_prncp_inv",
  "total_pymnt",
  "total_pymnt_inv",
  "total_rec_prncp",
  "total_rec_int",
  "total_rec_late_fee",
  "recoveries",
  "collection_recovery_fee",
  "last_pymnt_d",
  "last_pymnt_amnt",
  "last_credit_pull_d",
  "application_type"]
behaviour_var

In [ ]:
# let's now remove the behaviour variables from analysis
df = loan.drop(behaviour_var, axis=1)
df.info()

In [ ]:
# also, we will not be able to use the variables zip code, address, state etc.
# the variable 'title' is derived from the variable 'purpose'
# thus let get rid of all these variables as well

df = df.drop(['title', 'url', 'zip_code', 'addr_state'], axis=1)

In [ ]:
df['loan_status'] = df['loan_status'].astype('category')
df['loan_status'].value_counts()

In [ ]:
# filtering only fully paid or charged-off
df = df[df['loan_status'] != 'Current']
df['loan_status'] = df['loan_status'].apply(lambda x: 0 if x=='Fully Paid' else 1)

# converting loan_status to integer type
df['loan_status'] = df['loan_status'].apply(lambda x: pd.to_numeric(x))

# summarising the values
df['loan_status'].value_counts()

In [ ]:
# default rate
round(np.mean(df['loan_status']), 2)

The overall default rate is about 14%.

Let's first visualise the average default rates across categorical variables.

In [ ]:
# plotting default rates across grade of the loan
sns.barplot(x='grade', y='loan_status', data=df)
plt.show()

In [ ]:
# lets define a function to plot loan_status across categorical variables
def plot_cat(cat_var):
    sns.barplot(x=cat_var, y='loan_status', data=df)
    plt.show()
    

In [ ]:
# compare default rates across grade of loan
plot_cat('grade')

In [ ]:
# term: 60 months loans default more than 36 months loans
plot_cat('term')

In [ ]:
# sub-grade: as expected - A1 is better than A2 better than A3 and so on 
plt.figure(figsize=(16, 6))
plot_cat('sub_grade')

In [ ]:
# home ownership: not a great discriminator
plot_cat('home_ownership')

In [ ]:
# verification_status: surprisingly, verified loans default more than not verifiedb
plot_cat('verification_status')

In [ ]:
# purpose: small business loans defualt the most, then renewable energy and education
plt.figure(figsize=(16, 6))
plot_cat('purpose')

In [ ]:
# let's also observe the distribution of loans across years
# first lets convert the year column into datetime and then extract year and month from it
df['issue_d'].head()

In [ ]:
# extracting month and year from issue_date
df['month'] = df['issue_d'].apply(lambda x: x.month)
df['year'] = df['issue_d'].apply(lambda x: x.year)


In [ ]:
# let's first observe the number of loans granted across years
df.groupby('year').year.count()

In [ ]:
# number of loans across months
df.groupby('month').month.count()

In [ ]:
# lets compare the default rates across years
# the default rate had suddenly increased in 2011, inspite of reducing from 2008 till 2010
plot_cat('year')

In [ ]:
# comparing default rates across months: not much variation across months
plt.figure(figsize=(16, 6))
plot_cat('month')

In [ ]:
# loan amount: the median loan amount is around 10,000
sns.distplot(df['loan_amnt'])
plt.show()

In [ ]:
# binning loan amount
def loan_amount(n):
    if n < 5000:
        return 'low'
    elif n >=5000 and n < 15000:
        return 'medium'
    elif n >= 15000 and n < 25000:
        return 'high'
    else:
        return 'very high'
        
df['loan_amnt'] = df['loan_amnt'].apply(lambda x: loan_amount(x))


In [ ]:
df['loan_amnt'].value_counts()

In [ ]:
# let's compare the default rates across loan amount type
# higher the loan amount, higher the default rate
plot_cat('loan_amnt')

In [ ]:
# let's also convert funded amount invested to bins
df['funded_amnt_inv'] = df['funded_amnt_inv'].apply(lambda x: loan_amount(x))

In [ ]:
# funded amount invested
plot_cat('funded_amnt_inv')

In [ ]:
# lets also convert interest rate to low, medium, high
# binning loan amount
def int_rate(n):
    if len(n) <= 10:
        return 'low'
    elif len(n) > 10 and len(n) <=15:
        return 'medium'
    else:
        return 'high'
    
    
df['int_rate'] = df['int_rate'].apply(lambda x: int_rate(x))

In [ ]:
# comparing default rates across rates of interest
# high interest rates default more, as expected
plot_cat('int_rate')

In [ ]:
# debt to income ratio
def dti(n):
    if n <= 10:
        return 'low'
    elif n > 10 and n <=20:
        return 'medium'
    else:
        return 'high'
    

df['dti'] = df['dti'].apply(lambda x: dti(x))

In [ ]:
# comparing default rates across debt to income ratio
# high dti translates into higher default rates, as expected
plot_cat('dti')

In [ ]:
# comparing default rates across debt to income ratio
# high dti translates into higher default rates, as expected
plot_cat('dti')

In [ ]:
plot_cat('funded_amnt')


In [ ]:
# installment
def installment(n):
    if n <= 200:
        return 'low'
    elif n > 200 and n <=400:
        return 'medium'
    elif n > 400 and n <=600:
        return 'high'
    else:
        return 'very high'
    
df['installment'] = df['installment'].apply(lambda x: installment(x))

In [ ]:
# comparing default rates across installment
# the higher the installment amount, the higher the default rate
plot_cat('installment')

In [ ]:
# annual income
def annual_income(n):
    if n <= 50000:
        return 'low'
    elif n > 50000 and n <=100000:
        return 'medium'
    elif n > 100000 and n <=150000:
        return 'high'
    else:
        return 'very high'

df['annual_inc'] = df['annual_inc'].apply(lambda x: annual_income(x))

In [ ]:
# annual income and default rate
# lower the annual income, higher the default rate
plot_cat('annual_inc')

In [ ]:
# employment length
# first, let's drop the missing value observations in emp length
df = df[~df['emp_length'].isnull()]

# binning the variable
def emp_length(n):
    if n <= 1:
        return 'fresher'
    elif n > 1 and n <=3:
        return 'junior'
    elif n > 3 and n <=7:
        return 'senior'
    else:
        return 'expert'

df['emp_length'] = df['emp_length'].apply(lambda x: emp_length(x))

In [ ]:
# emp_length and default rate
# not much of a predictor of default
plot_cat('emp_length')

In [ ]:
# purpose: small business loans defualt the most, then renewable energy and education
plt.figure(figsize=(16, 6))
plot_cat('purpose')

In [ ]:
# lets first look at the number of loans for each type (purpose) of the loan
# most loans are debt consolidation (to repay otehr debts), then credit card, major purchase etc.
plt.figure(figsize=(16, 6))
sns.countplot(x='purpose', data=df)
plt.show()

In [ ]:
# filtering the df for the 4 types of loans mentioned above
main_purposes = ["credit_card","debt_consolidation","home_improvement","major_purchase"]
df = df[df['purpose'].isin(main_purposes)]
df['purpose'].value_counts()

In [ ]:
# plotting number of loans by purpose 
sns.countplot(x=df['purpose'])
plt.show()

In [ ]:
# let's now compare the default rates across two types of categorical variables
# purpose of loan (constant) and another categorical variable (which changes)

plt.figure(figsize=[10, 6])
sns.barplot(x='term', y="loan_status", hue='purpose', data=df)
plt.show()


In [ ]:
# lets write a function which takes a categorical variable and plots the default rate
# segmented by purpose 

def plot_segmented(cat_var):
    plt.figure(figsize=(10, 6))
    sns.barplot(x=cat_var, y='loan_status', hue='purpose', data=df)
    plt.show()

    
plot_segmented('term')

In [ ]:
# grade of loan
plot_segmented('grade')

In [ ]:
# home ownership
plot_segmented('home_ownership')

In [ ]:
# year
plot_segmented('year')

In [ ]:
# emp_length
plot_segmented('emp_length')

In [ ]:
# loan_amnt: same trend across loan purposes
plot_segmented('loan_amnt')

In [ ]:
# interest rate
plot_segmented('int_rate')

In [ ]:
# installment
plot_segmented('installment')

In [ ]:
# debt to income ratio
plot_segmented('dti')

In [ ]:
# annual income
plot_segmented('annual_inc')

In [ ]:
# variation of default rate across annual_inc
df.groupby('annual_inc').loan_status.mean().sort_values(ascending=False)

In [ ]:
# one can write a function which takes in a categorical variable and computed the average 
# default rate across the categories
# It can also compute the 'difference between the highest and the lowest default rate' across the 
# categories, which is a decent metric indicating the effect of the varaible on default rate

def diff_rate(cat_var):
    default_rates = df.groupby(cat_var).loan_status.mean().sort_values(ascending=False)
    return (round(default_rates, 2), round(default_rates[0] - default_rates[-1], 2))

default_rates, diff = diff_rate('annual_inc')
print(default_rates) 
print(diff)


In [ ]:
# filtering all the object type variables
df_categorical = df.loc[:, df.dtypes == object]
df_categorical['loan_status'] = df['loan_status']

# Now, for each variable, we can compute the incremental diff in default rates
print([i for i in df.columns])